# Modelo predictivo llamadas Contact Center
## Notebook 1 - Importación datos de envíos

## Importacion de librerias 

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import datetime
import calendar

pd.options.display.max_columns = None

### Importacion de los datos de envios por oficinas

##### Cargamos los datos de los envios desde el fichero shippings.csv

In [2]:
df = pd.read_csv('data/shippings.csv', low_memory=False)

In [3]:
df['Franquicia destino'].unique()

array([4800, 4802, 4803, 4806, 4810], dtype=int64)

In [4]:
df.columns

Index(['Unnamed: 0', 'Fecha envío', 'Número envío', 'Id. Fiscal',
       'Nombre Comercial', 'Código servicio', 'Nombre Rem', 'Población Rem',
       'CP Rem', 'Nombre vía Rem', 'Nombre', 'Población', 'Código postal',
       'Nombre vía', 'Total bultos', 'Franquicia origen', 'Franquicia destino',
       'Total', 'Estado', 'Tipo anomalía', 'Motivo', 'Importe Total'],
      dtype='object')

In [5]:
df = df.iloc[:, [1,5,16]]
df.head()

,Fecha envío,Código servicio,Franquicia destino
0,01/01/2019,Económico,4800
1,01/01/2019,Marítimo baleares,4800
2,01/01/2019,E-commerce,4800
3,01/01/2019,E-commerce,4800
4,01/01/2019,E-commerce,4800


In [6]:
df['Fecha envío'] = pd.to_datetime(df['Fecha envío'])

In [7]:
df[df['Fecha envío'].isnull()]

,Fecha envío,Código servicio,Franquicia destino


##### Obtenemos el dia de la semana para poder obtener un parámetro más para nuestro modelo

In [8]:
dias = list(calendar.day_name)

In [9]:
df['dayofweek'] = [dias[numero] for numero in df['Fecha envío'].dt.dayofweek]
df.head()

,Fecha envío,Código servicio,Franquicia destino,dayofweek
0,2019-01-01,Económico,4800,Tuesday
1,2019-01-01,Marítimo baleares,4800,Tuesday
2,2019-01-01,E-commerce,4800,Tuesday
3,2019-01-01,E-commerce,4800,Tuesday
4,2019-01-01,E-commerce,4800,Tuesday


In [10]:
df['Fecha envío'] =  pd.to_datetime(df['Fecha envío']).astype('str')

In [11]:
df = df[['Fecha envío','Código servicio','dayofweek']]
df.reset_index(inplace=True)
df = df[['Fecha envío','Código servicio','dayofweek']]
# Añadimos las columna 'total' inicializada a 1, para cuando agrupemos por fecha tener el total por día
df['total'] = 1

In [12]:
df.head()

,Fecha envío,Código servicio,dayofweek,total
0,2019-01-01,Económico,Tuesday,1
1,2019-01-01,Marítimo baleares,Tuesday,1
2,2019-01-01,E-commerce,Tuesday,1
3,2019-01-01,E-commerce,Tuesday,1
4,2019-01-01,E-commerce,Tuesday,1


##### Convertimos la columna del dia de la semana 'dayofweek' en 5 columnas donde representamos el dia de la semana como un vector

In [13]:
encoder = OneHotEncoder(handle_unknown='ignore')
enc = df[['dayofweek']]
encoder.fit(enc.values.reshape(-1,1))
encoded = encoder.transform(enc.values.reshape(-1,1))
encoded_df = pd.DataFrame(encoded.todense())
encoded_df.columns = encoder.categories_[0]

In [14]:
df = pd.concat([df,encoded_df], axis=1)

In [15]:
df.drop(columns=['Código servicio','dayofweek'], inplace=True)

##### Agrupamos por día/fecha para tener el total de envíos por fecha

In [16]:
df = df.groupby(['Fecha envío']).sum()

In [17]:
# Volvemos a normalizar el vector dia de la semana ya que cuando hemos realizado la agrupacion se ha sumado el valor del campo
df.Monday = [ 1 if value > 0 else 0 for value in df.Monday]
df.Tuesday = [ 1 if value > 0 else 0 for value in df.Tuesday]
df.Wednesday = [ 1 if value > 0 else 0 for value in df.Wednesday]
df.Thursday = [ 1 if value > 0 else 0 for value in df.Thursday]
df.Friday = [ 1 if value > 0 else 0 for value in df.Friday]
df.Saturday = [ 1 if value > 0 else 0 for value in df.Saturday]
df.Sunday = [ 1 if value > 0 else 0 for value in df.Sunday]
df.head()


,total,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
Fecha envío,,,,,,,,
2019-01-01,33,0,0,0,0,0,1,0
2019-01-02,1923,0,0,0,0,0,0,1
2019-01-03,1408,0,0,0,0,1,0,0
2019-01-04,3220,1,0,0,0,0,0,0
2019-01-05,23,0,0,1,0,0,0,0


In [18]:
df.columns

Index(['total', 'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday',
       'Tuesday', 'Wednesday'],
      dtype='object')

In [19]:
df.describe().iloc[0,0] ##Cantidad de registros

851.0

In [20]:
df = df.drop(labels=['Saturday','Sunday'], axis=1)

In [21]:
df.to_pickle('./data/df_Envios')